In [32]:
import string
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import nltk
import re
import seaborn as sns
mystem = Mystem()
morph = MorphAnalyzer()

In [51]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
%matplotlib inline  
pd.set_option('max_colwidth', 1000)

In [52]:
train_data = pd.read_csv('data/sentiment_twitter/train_sentiment_ttk.tsv', sep='\t')
test_data = pd.read_csv('data/sentiment_twitter/test_sentiment_ttk.tsv', sep='\t')

Обучаем текст, строим вектора.

In [53]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data.text.values)
X_train = tfidf.transform(train_data.text.values)
X_test = tfidf.transform(test_data.text.values)

In [54]:
y_train = train_data.label.values
y_test = test_data.label.values

В качестве классификатора - логистическая регрессия, не требующая нормализации текста

In [55]:
clf = LogisticRegression(penalty='l1')
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Предсказываем значения и оцениваем качество:

In [56]:
y_pred = clf.predict(X_test)

In [57]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.70      0.69      0.70       902
          0       0.66      0.76      0.71       972
          1       0.37      0.09      0.15       180

avg / total       0.65      0.67      0.65      2054

Макросредняя F1 мера -  0.5176166888561017
Микросредняя F1 мера -  0.6713729308666018


In [58]:
labels = clf.classes_
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(data=confusion_matrix(y_test, y_pred), annot=True, 
            fmt="d", xticklabels=labels, yticklabels=labels, ax=ax)
plt.title("Confusion matrix")
plt.show()

Теперь попробуем нормализовать текст. Вычистим знаки препинания, ссылки, упоминания аккаунтов в твиттере (реплаи) и хештеги.
Затем применим mystem и лемматизируем текст, затем склеивая все слова заново.

In [72]:
import html.parser

def normalize(text):
    """
    функция нормализации
    
    ::парметры::
    @text - ненормализованный текст (string)
    
    ::returns::
    нормализованный текст (string)
    """
    text = text.lower()
    text = html.parser.HTMLParser().unescape(text)
    #text = re.sub(r'<[^>]+>|\n', ' ', text)
    text = re.sub(r'@\w*|https?://.+|#.+|RT|\d+|[a-z]*', '', text)
    no_space_sequences_text = re.sub('  +', ' ', text)
    #text = html.unescape(text)
    punct_extended = string.punctuation + '«»—…“”'
    table = str.maketrans({ch: None for ch in punct_extended})
    
    text_tags = [word.translate(table) for word in text.split()]
    all_clear = ' '.join(text_tags)
    words_normalized = mystem.lemmatize(all_clear)
    stops = stopwords.words('russian')
    #words_normalized_no_stops = [word for word in words_normalized if word not in stops]
    l = [line.rstrip() for line in words_normalized]
    lemma_final = list(filter(None, l))
    return ' '.join(lemma_final)

Применяем функцию к тренировочной и тестовой выборкам:

In [73]:
train_data['normalized'] = train_data['text'].apply(normalize)
test_data['normalized'] = test_data['text'].apply(normalize)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  


TypeError: unsupported operand type(s) for -: 'str' and 'str'

Обучаем в векторайзере на нормализованных данных:

In [66]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data['normalized'].values)

X_train = tfidf.transform(train_data['normalized'].values)
X_test = tfidf.transform(test_data['normalized'].values)

In [67]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

Оценим качество теперь:

In [68]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.73      0.65      0.69       902
          0       0.65      0.81      0.72       972
          1       0.53      0.09      0.16       180

avg / total       0.67      0.68      0.66      2054

Макросредняя F1 мера -  0.5220039347841705
Микросредняя F1 мера -  0.676728334956183


Как видим, качество на макросредней мере выросло, а на микросредней - незначительно снизилось.

In [45]:
labels = clf.classes_
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(data=confusion_matrix(y_test, y_pred), annot=True, 
            fmt="d", xticklabels=labels, yticklabels=labels, ax=ax)
plt.title("Confusion matrix")
plt.show()

In [69]:
def print_important(vectorizer, clf, topn=10):
    features = vectorizer.get_feature_names()
    classes = clf.classes_
    importances = clf.coef_
    for i, cls in enumerate(classes):
        print('Значимые слова для класса - ', cls)
        important_words = sorted(list(zip(features, importances[i])), key=lambda x: abs(x[1]), reverse=True)[:topn]
        print([word for word,_ in important_words])
        print()

In [71]:
print_important(tfidf, clf, 10)

Значимые слова для класса -  -1
['не', 'гавно', 'изза', 'нет', 'крым', 'отключать', 'спасибо', 'сука', 'оштрафовать', 'запускать']

Значимые слова для класса -  0
['гавно', 'доллар', 'не', 'инструкция', 'для', 'настройка', 'изза', 'пожалуйста', 'ловить', 'любить']

Значимые слова для класса -  1
['узбекистан', 'спасибо', 'не', 'любить', 'запускать', 'хороший', 'зарабатывать', 'бесплатный', 'область', 'защита']

